In [1]:
import numpy as np
import pandas as pd
import pickle


In [2]:
import matplotlib.pyplot as plt

#from wordcloud import WordCloud
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [2]:
train = pd.read_csv(r'C:\Users\Shibbs\Desktop\Projects\Sentiment analysis\My dataset\Training\training_dataset.csv')

In [3]:
train['sentiment'].value_counts()

neutral      1194
worry        1187
sadness      1134
angry        1133
happiness    1121
Name: sentiment, dtype: int64

In [4]:
train.loc[:]

,Unnamed: 0,level_0,text,sentiment
0,0,0,charlene my love i miss you,sadness
1,1,1,the storm is here and the electricity is gone,sadness
2,2,2,so tired and i think im definitely going to ge...,sadness
3,3,3,on my way home n having deal w underage girls...,worry
4,4,4,im sorry people are so rude to you isaac they ...,sadness
...,...,...,...,...
5764,5764,5765,youre not irritating booooooooo love you omg...,angry
5765,5765,5766,controversy they causedyes we all know to expe...,angry
5766,5766,5767,thats so irritating i wish people cared about...,angry
5767,5767,5768,ricky sharma tweeted g aapne besharmi ki saar...,angry


In [6]:
train.to_csv('5emotions_final_processed_v2.csv')

In [5]:
embeddings_index = {}

f = open(r'C:\Users\Shibbs\Desktop\Projects\Sentiment analysis\My dataset\Embeddings\glove.twitter.27B.200d.txt', encoding="utf8")

for line in f:

    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')

GloVe data loaded


In [8]:
import re
train.text=train.text.astype(str)
## Iterate over the data to preprocess by removing stopwords
lines_without_stopwords=[] 
for line in train['text'].values: 
    line = line.lower()
    line_by_words = re.findall(r'(?:\w+)', line, flags = re.UNICODE) # remove punctuation ans split
    new_line=[]
    for word in line_by_words:
        if word not in stop:
            new_line.append(word)
    lines_without_stopwords.append(new_line)
texts = lines_without_stopwords

print(texts[0:5])

[['charlene', 'love', 'miss'], ['storm', 'electricity', 'gone'], ['tired', 'think', 'im', 'definitely', 'going', 'get', 'ear', 'infection', 'going', 'bed', 'early'], ['way', 'home', 'n', 'deal', 'w', 'underage', 'girls', 'drinking', 'gin', 'da', 'bus', 'talking', 'bout', 'keggers', 'feel', 'old'], ['im', 'sorry', 'people', 'rude', 'isaac', 'get', 'manners', 'know', 'better', 'lewd']]


In [6]:
import re
train.text=train.text.astype(str)
## Iterate over the data to preprocess by removing stopwords
new_line=[] 
for line in train['text'].values: 
    line = line.lower()
    line_by_words = re.findall(r'(?:\w+)', line, flags = re.UNICODE) # remove punctuation ans split
    new_line.append(line_by_words)
    
texts = new_line

print(texts[0:5])

[['charlene', 'my', 'love', 'i', 'miss', 'you'], ['the', 'storm', 'is', 'here', 'and', 'the', 'electricity', 'is', 'gone'], ['so', 'tired', 'and', 'i', 'think', 'im', 'definitely', 'going', 'to', 'get', 'an', 'ear', 'infection', 'going', 'to', 'bed', 'early', 'for', 'once'], ['on', 'my', 'way', 'home', 'n', 'having', 'deal', 'w', 'underage', 'girls', 'drinking', 'gin', 'on', 'da', 'bus', 'while', 'talking', 'bout', 'keggers', 'i', 'feel', 'old'], ['im', 'sorry', 'people', 'are', 'so', 'rude', 'to', 'you', 'isaac', 'they', 'should', 'get', 'some', 'manners', 'and', 'know', 'better', 'than', 'to', 'be', 'so', 'lewd']]


In [7]:
with open('list.pkl', 'wb') as f:
    pickle.dump(texts, f)

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_NUM_WORDS = 30000
MAX_SEQUENCE_LENGTH = len(max(texts))*2
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

tokenizer.fit_on_texts(texts)
pickle.dump(tokenizer, open('tokenizer.pickle', 'wb'))
sequences = tokenizer.texts_to_sequences(texts)
'''pickle.dump(sequences, open('sequences.pickle', 'wb'))'''

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

labels = pd.get_dummies(train['sentiment']).values
print(data.shape)
print(labels.shape)

Using TensorFlow backend.


Found 11594 unique tokens.
(5769, 48)
(5769, 5)


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size= 0.20)

In [10]:
from keras.layers import Embedding
from keras.initializers import Constant

## EMBEDDING_DIM =  ## seems to need to match the embeddings_index dimension
EMBEDDING_DIM = embeddings_index.get('a').shape[0]
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [11]:
len(word_index)

11594

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, GRU, Conv1D, MaxPooling1D, Dropout, SpatialDropout1D, Activation, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam
add = Adam(lr = 0.001)

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length = MAX_SEQUENCE_LENGTH, weights = [embedding_matrix], trainable = False))

model.add(SpatialDropout1D(0.4))



'''model.add(Dropout(0.5))'''
model.add((LSTM(128 , return_sequences = True, dropout = 0.5 , recurrent_dropout = 0.5)))

'''model.add(MaxPooling1D(pool_size=2))'''
'''model.add(Conv1D(100, 7, activation='relu'))'''
'''model.add(GlobalMaxPooling1D())'''
'''model.add(Conv1D(100, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))'''
'''

model.add(Dense(50,activation = 'relu'))
model.add(Dropout(0.5))
'''
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(5, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer=add, metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 200)           2319000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 48, 64)            67840     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 44, 128)           41088     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 2,428,573
Trainable params: 109,573
Non-trainable params: 2,319,000
______________________________________

In [13]:
epochs = 100
batch_size = 128

model.fit(x_train, y_train,validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[
                    EarlyStopping(monitor='val_loss', patience=2, min_delta=0.0001)])

Train on 4615 samples, validate on 1154 samples
Epoch 1/100
4615/4615 [==============================] - 34s 7ms/step - loss: 0.4793 - accuracy: 0.8044 - val_loss: 0.4249 - val_accuracy: 0.8194
Epoch 2/100
4615/4615 [==============================] - 6s 1ms/step - loss: 0.4200 - accuracy: 0.8233 - val_loss: 0.3853 - val_accuracy: 0.8282
Epoch 3/100
4615/4615 [==============================] - 6s 1ms/step - loss: 0.3927 - accuracy: 0.8299 - val_loss: 0.3705 - val_accuracy: 0.8326
Epoch 4/100
4615/4615 [==============================] - 6s 1ms/step - loss: 0.3840 - accuracy: 0.8312 - val_loss: 0.3615 - val_accuracy: 0.8359
Epoch 5/100
4615/4615 [==============================] - 6s 1ms/step - loss: 0.3744 - accuracy: 0.8335 - val_loss: 0.3477 - val_accuracy: 0.8378
Epoch 6/100
4615/4615 [==============================] - 6s 1ms/step - loss: 0.3665 - accuracy: 0.8358 - val_loss: 0.3452 - val_accuracy: 0.8393
Epoch 7/100
4615/4615 [==============================] - 6s 1ms/step - loss: 0.36

In [16]:
with open('model.pickle', 'wb') as file:  
    pickle.dump(model, file)

In [14]:
data = pad_sequences(tokenizer.texts_to_sequences(['love it so much']),padding = 'post', truncating='post',maxlen=48)
score = model.predict([data])

In [15]:
score

array([[0.00415167, 0.32623369, 0.16274385, 0.27080914, 0.23606166]],
      dtype=float32)

In [54]:
data

array([[172,  80,  99,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [101]:
def prediction(dataframe):
    
    list1 = []
    list2 = []
    preds = []
    list3 = []
    
    dataframe.tweet = dataframe.tweet.astype(str)
    for index,row in dataframe.iterrows():
        desc = row['tweet']
        list1.append(desc)
    
    '''list2 = pad_sequences(tokenizer.texts_to_sequences([list1]),padding = 'post', truncating='post',maxlen=MAX_SEQUENCE_LENGTH)
    score = model.predict([list2])'''
        
    for i in list1:
        
        list2 = pad_sequences(tokenizer.texts_to_sequences([i]),padding = 'post', truncating='post',maxlen=MAX_SEQUENCE_LENGTH)
        score = model.predict([list2])
        preds.append(score)
        emotions = np.array(preds)
        
    for i in range(0, len(emotions)):
        if np.argmax(emotions[i][0]) == 0:
            list3.append('anger')
        elif np.argmax(emotions[i][0]) == 1:
            list3.append('happiness')
        elif np.argmax(emotions[i][0]) == 2:
            list3.append('neutral')
        elif np.argmax(emotions[i][0]) == 3:  
            list3.append('sadness')
        elif np.argmax(emotions[i][0]) == 4:
            list3.append('worry')
        
        
    
    return list3

In [102]:
sentiments = prediction(df0)

In [103]:
df0['sentiment'] = sentiments

In [104]:
df0['sentiment'].value_counts()

neutral      149
worry         99
sadness       68
happiness     41
anger          1
Name: sentiment, dtype: int64

In [105]:
index = 11

print(df0['tweet'][index])
print(df0['sentiment'][index])

the city of dreams slowly is coming back to life after a hiatus like never before slowly but surely we shall experience mumbai in all its former characteristic glory here s saluting the patience of the people the true lifeline of mumbai mumbai unlock
worry


In [337]:
df0.to_csv(r'C:\Users\Shibbs\Desktop\Projects\Sentiment analysis\My dataset\glove_bi_lstm7_7.3.csv')

In [45]:
model.save('glove_bi_lstm_gpu9_7.0.h5')

In [46]:
model.save_weights('weights_glove_bi_lstm_gpu9_7.0.h5')

In [170]:
#df1 =  pd.read_csv(r'C:\Users\Shibbs\Desktop\Projects\Sentiment analysis\My dataset\glove_cnn_lstm2_7.2.csv')

In [173]:
df1['sentiment'].value_counts()

neutral      189
worry         73
sadness       52
happiness     26
joy           18
Name: sentiment, dtype: int64

In [172]:
for index, row in df0.iterrows():
    if df0['sentiment'][index] == 'happiness':
        print(df0['tweet'][index])
        

seeing the current covid india situation and talking abt unlock in mumbai is surprising new cases have not stopped no other city wld have dreamed to even unlock in current situation mumbai shd have been last in country to open up
a map prepared by csir neeri mumbai showing covid cases in india th june coronavirus maharashtrafightscorona medicalmaharashtra medd coronaupdatesindia unlockingindia unlock mumbaifightscovid stayhomestaysafesavelives covid india unlockindia
i feel people from other states are getting back to mumbai after unlock pls ensure to follow protocol so that it doesn t spread again
how many covid tests are done on a daily basis in mumbai thane kalyan dombivli ambarnath badlapur vasai virar navi mumbai why is there unlock when there are less number beds hospital facilities covid mumbai udhavthackeray unlock lockdownfailed
the city of dreams slowly is coming back to life after a hiatus like never before slowly but surely we shall experience mumbai in all its former chara